## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> add any cells to this Jupyter Notebook, because that will crash the autograder.
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> implement any additional libraries into this workbook.
</div>

All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  

<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`bucket()`

`long_trips()`

`manhattan_trips()`

`weighted_profit()`

`final_output()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from the NYC-TLC dataset during auto-grading.

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove or modify the following utility functions:
</div>

`load_data()`

`main()`

<div class="alert alert-block alert-info">
    Do <strong>not</strong> change the below cell. Run it to initialize your PySpark instance. If you don't get any output, make sure your Notebook's Kernel is set to "PySpark" in the top right corner.
</div>

In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1616470090737_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remodify the below cell. It contains the function for loading data and all imports, and the function for running your code.
</div>

In [2]:
#### DO NOT CHANGE ANYTHING IN THIS CELL ####

from pyspark.sql.functions import col
from pyspark.sql import *

def load_data(size='small'):
    # Loads the data for this question. Do not change this function.
    # This function should only be called with the parameter 'small' or 'large'
    
    if size != 'small' and size != 'large':
        print("Invalid size parameter provided. Use only 'small' or 'large'.")
        return
    
    input_bucket = "s3://cse6242-spring2021"
    
    # Load Trip Data
    trip_path = '/'+size+'/yellow_tripdata*'
    trips = spark.read.csv(input_bucket + trip_path, header=True, inferSchema=True)
    print("Trip Count: ",trips.count()) # Prints # of trips (# of records, as each record is one trip)
    
    # Load Lookup Data
    lookup_path = '/'+size+'/taxi*'
    lookup = spark.read.csv(input_bucket + lookup_path, header=True, inferSchema=True)
    
    return trips, lookup

def main(size, bucket):
    # Runs your functions implemented above.
    
    print(user())
    trips, lookup = load_data(size=size)
    trips = long_trips(trips)
    mtrips = manhattan_trips(trips, lookup)
    wp = weighted_profit(trips, mtrips)
    final = final_output(wp,lookup)
    
    # Outputs the results for you to visually see
    final.show()
    
    # Writes out as a CSV to your bucket.
    final.write.csv(bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Implement the below functions for this assignment:
<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> change any function inputs or outputs, and ensure that the dataframes your code returns align with the schema definitions commented in each function
</div>

## 3a. [1 pt] Update the `user()` function
This function should return your GT username, eg: gburdell3

In [3]:
def user():
    # Returns a string consisting of your GT username.
    return 'pkubsad3'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3b. [2 pts] Update the `long_trips()` function
This function filters trips to keep only trips greater than or equal to 2 miles.

In [4]:
def long_trips(trips):
    # Returns a Dataframe with Schema the same as :trips:
    print("in long_trips")
    return trips.filter(col("trip_distance")>=2.0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3c. [6 pts] Update the `manhattan_trips()` function

This function determines the top 20 locations with a `DOLocationID` in manhattan by passenger_count (pcount).

Example output formatting:

```
+--------------+--------+
| DOLocationID | pcount |
+--------------+--------+
|             5|      15|
|            16|      12| 
+--------------+--------+
```

In [5]:
from pyspark.sql.functions import col,count,sum
def manhattan_trips(trips, lookup):
    # Returns a Dataframe with Schema: DOLocationID, pcount
    print("in manhattan trips")
    joinCondition = [trips.DOLocationID==lookup.LocationID,lookup.Borough=="Manhattan"]
    temp=trips.alias("trips").join(lookup.alias("lookup"),joinCondition,"inner")\
    .groupBy("DOLocationId")\
    .agg(sum(col("passenger_count")).alias("pcount"))\
    .orderBy(col("pcount").desc(),col("DOLocationID"))\
    .limit(20)
    temp.show()
    print(type(temp))
    print((temp.count(), len(temp.columns)))
    return temp

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3d. [6 pts] Update the `weighted_profit()` function
This function should determine the average `total_amount`, the total count of trips, and the total count of trips ending in the top 20 destinations and return the `weighted_profit` as discussed in the homework document.

Example output formatting:
```
+--------------+-------------------+
| PULocationID |  weighted_profit  |
+--------------+-------------------+
|            18| 33.784444421924436| 
|            12| 21.124577637149223| 
+--------------+-------------------+
```

In [6]:
from pyspark.sql.functions import avg
def weighted_profit(trips, mtrips): 
    # Returns a Dataframe with Schema: PULocationID, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    temp=trips.groupBy(col("PULocationID").alias("PULocationID")).agg(avg(col("total_amount")).alias("avg_total_amount"),count("*").alias("count"))
    temp2=trips.alias("trips").join(mtrips.alias("mtrips"),col("trips.DOLocationID")==col("mtrips.DOLocationID")).groupBy(col("trips.PULocationID")).agg(count("*").alias("top_count"))
    temp3=temp.join(temp2, temp["PULocationID"]==temp2["PULocationID"])
    temp4=temp3.select("trips.PULocationID",((col("top_count")/col("count"))*col("avg_total_amount")).alias("weighted_profit"))
    temp4.show()
    print(type(temp4))
    print((temp4.count(), len(temp4.columns)))
    print(type(temp4))
    return temp4

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3e. [5 pts] Update the `final_output()` function
This function will take the results of `weighted_profit`, links it to the `borough` and `zone` and returns the top 20 locations with the highest `weighted_profit`.

Example output formatting:
```
+------------+---------+-------------------+
|    Zone    | Borough |  weighted_profit  |
+----------------------+-------------------+
| JFK Airport|   Queens|  16.95897820117925|
|     Jamaica|   Queens| 14.879835188762488|
+------------+---------+-------------------+
```

In [7]:
def final_output(calc, lookup): 
    # Returns a Dataframe with Schema: Zone, Borough, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    temp=calc.orderBy(col("weighted_profit").desc()).limit(20)
    temp2=temp.alias("calc").join(lookup.alias("lookup"),col("calc.PULocationID")==col("lookup.LocationID")).orderBy(col("weighted_profit").desc()).select("Zone","Borough","weighted_profit")
    temp2=temp2.coalesce(1)
    temp2.show()
    print(type(temp2))
    print((temp2.count(), len(temp2.columns)))
#     print(temp2.count())
    return temp2

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<div class="alert alert-block alert-info">
    Test your code on the small dataset first, as the large dataset will take a significantly longer time to run
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> use the same bucket url for multiple runs of the `main()` function, as this will cause errors. Make sure to change the name of your output location every time. (ie: s3://cse6242-gburdell3/output-small2)
</div>

Update the below cell with the path to your bucket, then run the below cell to run your code to store the results in S3.

When you have confirmed the results of the small dataset, run it again using the large dataset. Your output file will appear in a folder in your s3 bucket called YOUROUTPUT.csv as a csv file with a name something like part-0000-4d992f7a-0ad3-48f8-8c72-0022984e4b50-c000.csv. Download this file and rename it to q3_output.csv for submission. Do not make any other changes to the file. 

In [8]:
# update your bucket path
# bucket = 's3://cse6242-pkubsad3/output-small'
# main('small', bucket)

bucket = 's3://cse6242-pkubsad3/output-large'
main('large', bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

pkubsad3
Trip Count:  187203269
in long_trips
in manhattan trips
+------------+---------+
|DOLocationId|   pcount|
+------------+---------+
|         236|3528776.0|
|         230|3162933.0|
|         161|3116127.0|
|         162|2714476.0|
|          48|2678151.0|
|         239|2595985.0|
|         170|2529140.0|
|         231|2502147.0|
|          79|2495986.0|
|         238|2423210.0|
|         142|2386885.0|
|         237|2330464.0|
|         263|2285207.0|
|         141|2190292.0|
|          68|2182082.0|
|         186|2104706.0|
|          13|2064024.0|
|         163|2041180.0|
|         246|2005676.0|
|         140|2000394.0|
+------------+---------+

<class 'pyspark.sql.dataframe.DataFrame'>
(20, 2)
+------------+------------------+
|PULocationID|   weighted_profit|
+------------+------------------+
|           7| 6.632736951569291|
|          31| 9.157055818578993|
|          18|2.6927146799551323|
|          61| 2.545987882624299|
|          75| 9.785059740280246|
|          2

#### Testing

<div class="alert alert-block alert-info">
    You may use the below cell for any additional testing you need to do, however any code implemented below will not be run or used when grading
</div>

In [ ]:
bucket = 's3://cse6242-pkubsad3/output-small'
trips,lookup=load_data("small")

In [ ]:
# lookup.show()
# trips.show()

In [ ]:
long_trips=long_trips(trips)

In [ ]:
mtrips = manhattan_trips(long_trips, lookup)

In [ ]:
ret=weighted_profit(long_trips,mtrips)

In [ ]:
final_output(ret,lookup)